## Interactive Dashboard for filtering and downloading data

 In this code, you will find an **interactive dashboard** with a step-by-step guide for its proper development and use. While it is designed for a company, it can easily be modified to suit your specific needs. The dashboard allows a user with limited technical knowledge to easily visualize and filter data. Additionally, it provides the option to **download a .csv file** containing the **filtered results**.

 I highly recommend saving this code as a .py file and using it as a module where all the functions are defined. Then, you can run it separately as a program. This way, your colleague or colleagues will only need to run the following code:

 *from dashboard import create_dashboard*
 
 *app = create_dashboard()*
 
 *app.run_server(debug=True)*

 To make this work, you'll need to make a few small adjustments to the code, but it's nothing complicated. Simply save the code as dashboard.py and import create_dashboard so that the program can be executed.


In [ ]:
# !pip install dash plotly pandas

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Input, Output, State
import dash
import base64
import io

# Load data
df = pd.read_csv("ventas_azulejos.csv")
df["Fecha"] = pd.to_datetime(df["Fecha"])

# Create app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1("Interactive Dashboard - Tile Company", style={'textAlign': 'center'}),

    html.Div([
        html.Div([
            html.Label("Select Products:"),
            dcc.Dropdown(
                id='producto-dropdown',
                options=[{'label': p, 'value': p} for p in df['Producto'].unique()],
                value=df['Producto'].unique().tolist(),
                multi=True
            ),
        ], style={'width': '45%', 'display': 'inline-block'}),

        html.Div([
            html.Label("Date Range:"),
            dcc.DatePickerRange(
                id='fecha-range',
                start_date=df["Fecha"].min(),
                end_date=df["Fecha"].max()
            )
        ], style={'width': '45%', 'float': 'right', 'display': 'inline-block'}),
    ], style={'padding': '20px'}),

    html.Div(id='kpi-container', style={'display': 'flex', 'justifyContent': 'space-around', 'margin': '20px'}),

    dcc.Graph(id='grafico-animado'),
    dcc.Graph(id='grafico-cantidad'),

    html.Div([
        html.Button("📥 Download Filtered Data", id="btn-download"),
        dcc.Download(id="download-dataframe-csv")
    ], style={'textAlign': 'center', 'margin': '20px'}),
])

# Callback to update KPIs and charts
@app.callback(
    [Output('grafico-animado', 'figure'),
     Output('grafico-cantidad', 'figure'),
     Output('kpi-container', 'children')],
    [Input('producto-dropdown', 'value'),
     Input('fecha-range', 'start_date'),
     Input('fecha-range', 'end_date')]
)
def update_dashboard(products, start_date, end_date):
    df_filtered = df[
        (df['Producto'].isin(products)) &
        (df['Fecha'] >= pd.to_datetime(start_date)) &
        (df['Fecha'] <= pd.to_datetime(end_date))
    ]

    # KPIs
    total_sales = df_filtered['Total'].sum()
    total_quantity = df_filtered['Cantidad'].sum()
    top_city = df_filtered.groupby("Ciudad")["Total"].sum().idxmax() if not df_filtered.empty else "N/A"

    kpis = [
        html.Div([
            html.H4("Total Sales:"),
            html.P(f"${total_sales:,.2f}")
        ], style={'textAlign': 'center'}),

        html.Div([
            html.H4("Total Quantity:"),
            html.P(f"{total_quantity} units")
        ], style={'textAlign': 'center'}),

        html.Div([
            html.H4("Top City:"),
            html.P(top_city)
        ], style={'textAlign': 'center'})
    ]

    # Animated Chart
    fig_animado = px.bar(
        df_filtered,
        x='Ciudad',
        y='Total',
        color='Producto',
        animation_frame=df_filtered["Fecha"].dt.strftime('%Y-%m'),
        animation_group='Sucursal',
        title='Sales by City and Product',
        range_y=[0, df["Total"].max() + 200] if not df_filtered.empty else [0, 1000]
    )

    # Quantity Chart
    fig_quantity = px.line(
        df_filtered.groupby(["Fecha", "Producto"]).sum(numeric_only=True).reset_index(),
        x="Fecha",
        y="Cantidad",
        color="Producto",
        markers=True,
        title="Quantity of Products Sold by Date"
    )

    return fig_animado, fig_quantity, kpis

# Callback to handle the CSV download
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("btn-download", "n_clicks"),
    State('producto-dropdown', 'value'),
    State('fecha-range', 'start_date'),
    State('fecha-range', 'end_date'),
    prevent_initial_call=True,
)
def download_filtered_data(n_clicks, products, start_date, end_date):
    if n_clicks is None:  # Ensure that it only triggers on click
        return dash.no_update

    # Filter data based on dropdown and date range
    df_filtered = df[
        (df['Producto'].isin(products)) &
        (df['Fecha'] >= pd.to_datetime(start_date)) &
        (df['Fecha'] <= pd.to_datetime(end_date))
    ]
    
    # Return the DataFrame directly to `dcc.send_data_frame` to generate CSV
    return dcc.send_data_frame(df_filtered.to_csv, "filtered_data.csv", index=False)

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
**AGOSTINA MARINI STASI**

Github: *https://github.com/agostinamarini*

LinkedIn: *www.linkedin.com/in/agostina-marini-1b18a31b3*